<a href="https://colab.research.google.com/github/SriVinayA/SJSU-CMPE297-SpecialTopics/blob/main/assignment_3_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Unsloth and required dependencies
!pip install -q unsloth
!pip uninstall -q unsloth -y && pip install -q --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

In [2]:
# Import necessary libraries
from unsloth import FastLanguageModel
import torch
from transformers import TrainingArguments
from unsloth import UnslothTrainer, UnslothTrainingArguments, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Define some essential configurations
max_seq_length = 2048  # Maximum sequence length for the model
dtype = None  # Auto-detection of data type (e.g., Float16 or BFloat16)
load_in_4bit = True  # Use 4-bit quantization to reduce memory usage

In [4]:
# Load a pretrained base model using Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",  # Base model to start with
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2024.11.10: Fast Mistral patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [5]:
# Add LoRA adapters for further training
model = FastLanguageModel.get_peft_model(
    model,
    r=128,  # Low-Rank Adaptation rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj",
                    "up_proj", "down_proj", "lm_head", "embed_tokens"],
    lora_alpha=32,  # Scaling factor for LoRA layers
    lora_dropout=0,  # Dropout rate for LoRA layers
    use_gradient_checkpointing="unsloth",  # Optimized gradient checkpointing
    use_rslora=True,  # Use Rank-Stabilized LoRA for improved stability
)

print("Setup complete. Model and tokenizer are ready for continued pretraining.")

Unsloth: Offloading input_embeddings to disk to save VRAM


/usr/local/lib/python3.10/dist-packages/unsloth/models/_utils.py:747: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename, map_location = "cpu", 

Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM
Setup complete. Model and tokenizer are ready for continued pretraining.


In [14]:
import pandas as pd
from datasets import Dataset

# Enhanced synthetic dataset with instruction-response pairs
synthetic_data = [
    {"instruction": "Saludar", "response": "Hola, ¿cómo estás?"},
    {"instruction": "Despedirse", "response": "Adiós, que tengas un buen día."},
    {"instruction": "Presentarse", "response": "Me llamo ChatGPT y estoy aprendiendo español."},
    {"instruction": "Hacer una pregunta", "response": "¿Cuál es tu comida favorita?"},
    {"instruction": "Responder a una pregunta", "response": "Mi comida favorita es la paella."},
    {"instruction": "Explicar qué es un saludo", "response": "Un saludo es una manera de expresar cortesía o amistad, como decir 'hola' o 'buenos días'."},
    {"instruction": "Describir la tierra", "response": "La Tierra es el tercer planeta del sistema solar y el único conocido que alberga vida."},
    {"instruction": "Explicar qué es la música", "response": "La música es una forma de arte que utiliza sonidos y silencios organizados en el tiempo."},
    {"instruction": "Contar un chiste", "response": "¿Por qué el libro de matemáticas se deprimió? Porque tenía demasiados problemas."},
    {"instruction": "Dar una recomendación", "response": "Te recomiendo leer 'Cien Años de Soledad' de Gabriel García Márquez."},
]

In [15]:
# Define a prompt template for instruction-response pairs
instruction_response_prompt = """### Instrucción:
{}

### Respuesta:
{}"""

EOS_TOKEN = tokenizer.eos_token  # End-of-Sequence token

def formatting_prompts_func_enhanced(examples):
    instructions = examples["instruction"]
    responses = examples["response"]
    outputs = []
    for instr, resp in zip(instructions, responses):
        # Format each example with the instruction and response
        formatted_text = instruction_response_prompt.format(instr, resp) + EOS_TOKEN
        outputs.append(formatted_text)
    return {"text": outputs}

# Create a Hugging Face Dataset from the enhanced synthetic data
df_synthetic = pd.DataFrame(synthetic_data)
dataset = Dataset.from_pandas(df_synthetic)

# Apply the enhanced formatting function to the dataset
dataset = dataset.map(formatting_prompts_func_enhanced, batched=True)

print("Enhanced synthetic dataset is ready for training.")
print(dataset)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Enhanced synthetic dataset is ready for training.
Dataset({
    features: ['instruction', 'response', 'text'],
    num_rows: 10
})


In [16]:
# Configure updated training arguments
training_args = UnslothTrainingArguments(
    per_device_train_batch_size=2,  # Increased batch size due to more data
    gradient_accumulation_steps=2,  # Adjusted for efficiency
    max_steps=200,  # Increased steps to allow better learning
    warmup_steps=20,  # More warmup steps for stability
    learning_rate=3e-5,  # Slightly reduced learning rate
    embedding_learning_rate=1e-5,  # Maintain smaller learning rate for embeddings
    fp16=not is_bfloat16_supported(),  # Use FP16 if BFloat16 isn't supported
    bf16=is_bfloat16_supported(),  # Use BFloat16 if supported
    logging_steps=10,  # Log progress every 10 steps
    weight_decay=0.01,  # Regularization to prevent overfitting
    lr_scheduler_type="linear",  # Linear learning rate decay
    seed=42,  # Reproducibility
    output_dir="outputs",  # Directory to save outputs
)

In [17]:
# Initialize the trainer with the enhanced dataset and updated training arguments
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,  # Use the enhanced synthetic dataset
    dataset_text_field="text",  # The field containing formatted text
    max_seq_length=max_seq_length,  # Maximum sequence length
    dataset_num_proc=1,  # Single processor for simplicity
    args=training_args,
)

# Start training
print("Starting continued pretraining with enhanced dataset...")
trainer_stats = trainer.train()

# Output training statistics
print("Training complete.")
print(trainer_stats)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Starting continued pretraining with enhanced dataset...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 200
 "-____-"     Number of trainable parameters = 597,688,320


Unsloth: Setting lr = 1.00e-05 instead of 3.00e-05 for embed_tokens.
Unsloth: Setting lr = 1.00e-05 instead of 3.00e-05 for lm_head.


Step,Training Loss
10,3.702400
20,3.708100
30,3.718000
40,3.140300
50,1.917800
60,1.501300
70,1.232300
80,1.047400
90,0.926400
100,0.814400


Training complete.
TrainOutput(global_step=200, training_loss=1.357484917640686, metrics={'train_runtime': 387.9433, 'train_samples_per_second': 2.062, 'train_steps_per_second': 0.516, 'total_flos': 1660969299492864.0, 'train_loss': 1.357484917640686, 'epoch': 80.0})


In [18]:
# Prepare a test instruction in Spanish
test_instruction = "Explica qué es un saludo en español."

# Define the prompt using the instruction
test_prompt = f"""### Instrucción:
{test_instruction}

### Respuesta:"""

# Tokenize the input for the model
inputs = tokenizer(
    [test_prompt],
    return_tensors="pt",
    truncation=True,
    max_length=max_seq_length
).to("cuda")

# Enable optimized inference
FastLanguageModel.for_inference(model)

# Generate a response using the model with adjusted parameters
output = model.generate(
    **inputs,
    max_new_tokens=50,  # Limit the length of the generated text
    temperature=0.7,  # Introduce some randomness to the generation
    top_k=50,  # Limit to top 50 probable next tokens
    top_p=0.95,  # Nucleus sampling for more diverse outputs
    use_cache=True,  # Speed up generation with caching
)

# Decode and display the generated response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("\nGenerated Response:")
print(response)



Generated Response:
### Instrucción:
Explica qué es un saludo en español.

### Respuesta:
Un saludo es una manera de expresar cortesía o amistad, como decir 'hola' o 'buenos días'.
